In [36]:
import pandas as pd

def extract_team_nickname(full_name):
    return full_name.split()[-1]

# Load the CSV files
game_stats_path = './game_stats.csv'
predictions_path = './predictions.csv'

# Read the CSV files into pandas DataFrames
game_stats_df = pd.read_csv(game_stats_path)
predictions_df = pd.read_csv(predictions_path)

# Standardize the date format in both DataFrames
game_stats_df['game_date'] = pd.to_datetime(game_stats_df['game_date']).dt.date
predictions_df['date'] = pd.to_datetime(predictions_df['date']).dt.date

# Apply the nickname extraction function
game_stats_df['team_full_name_home'] = game_stats_df['team_full_name_home'].apply(extract_team_nickname)
game_stats_df['team_full_name_visitor'] = game_stats_df['team_full_name_visitor'].apply(extract_team_nickname)

# Adding a column for the actual winner in predictions_df
predictions_df['actual_winner'] = None

# Iterate through predictions_df and update with actual winner
for index, row in predictions_df.iterrows():
    # Extract prediction details
    prediction_date = row['date']
    home_team = row['Home Team']  # Assumes this is a nickname
    visitor_team = row['Visitor Team']  # Assumes this is a nickname

    # Find the corresponding game in game_stats_df
    game = game_stats_df[(game_stats_df['game_date'] == prediction_date) &
                         (game_stats_df['team_full_name_home'].str.lower().str.contains(home_team.lower())) &
                         (game_stats_df['team_full_name_visitor'].str.lower().str.contains(visitor_team.lower()))]

    if not game.empty:
        # Determine whether the home or visitor team won
        if game.iloc[0]['game_home_team_score'] > game.iloc[0]['game_visitor_team_score']:
            winning_team = home_team
        else:
            winning_team = visitor_team

        # Update the actual_winner in predictions_df
        predictions_df.at[index, 'actual_winner'] = winning_team
    else:
        print(f"No matching game found for date: {prediction_date}, Home Team: {home_team}, Visitor Team: {visitor_team}")

# Save the updated DataFrame, overwriting the existing predictions.csv file
predictions_df.to_csv(predictions_path, index=False)

# Display the first few rows of the updated predictions DataFrame for verification
print(predictions_df.head())


         date Start (ET) Visitor Team  PTS Home Team  PTS.1  Unnamed: 6  \
0  2023-12-11      7:00p         Heat  NaN   Hornets    NaN         NaN   
1  2023-12-11      7:00p       Pacers  NaN   Pistons    NaN         NaN   
2  2023-12-11      7:00p    Cavaliers  NaN     Magic    NaN         NaN   
3  2023-12-11      7:00p      Wizards  NaN     76ers    NaN         NaN   
4  2023-12-11      7:30p      Nuggets  NaN     Hawks    NaN         NaN   

   Unnamed: 7  Attend.                 Arena  Notes predicted_winner  \
0         NaN      NaN       Spectrum Center    NaN             Heat   
1         NaN      NaN  Little Caesars Arena    NaN           Pacers   
2         NaN      NaN          Amway Center    NaN            Magic   
3         NaN      NaN    Wells Fargo Center    NaN            76ers   
4         NaN      NaN      State Farm Arena    NaN            Hawks   

   win_probability actual_winner  
0         0.653512          Heat  
1         0.672488        Pacers  
2         0